In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import scipy.signal as sg
import scipy as sp
from mpl_toolkits import mplot3d
import ellipses as el
from matplotlib.patches import Ellipse
import sys
%matplotlib inline
# helper functions

# ellipse fitting function
def efit(data, maxpR, right=True):
    lsqe = el.LSqEllipse()
    lsqe.fit(data)
    center, width, height, phi = lsqe.parameters()
    vector = [(center[0]-maxpR[1]), (center[1]-maxpR[0])]
    vecmag = np.sqrt(vector[0]**2+vector[1]**1)
    return center, vector

# remove noise
def debubble(cpos, cneg, rpos, rneg, win_):
#     input: xcor of edge points
#     ncol: number of column
#     posfirst: posedge happens before negedge
    counts = np.bincount(cpos)
    pmode = np.argmax(counts)
    counts2 = np.bincount(cneg)
    nmode = np.argmax(counts2)
    ta1 = np.zeros(len(cpos)).astype('bool')
    ta2 = np.zeros(len(cneg)).astype('bool')
    print('truncation: ', pmode, '+-', win_)
    print('truncation: ', nmode, '+-', win_)
    for i in range(len(cpos)):
        ta1[i] = (cpos<(pmode+win_))[i] and (cpos>(pmode-win_))[i]
    for i in range(len(cneg)):
        ta2[i] = (cneg<(nmode+win_))[i] and (cneg>(nmode-win_))[i]
    return cpos[ta1], cneg[ta2], rpos[ta1], rneg[ta2]

# imgEye : list of 2d array, image
# frame: 2d array
# facecor: (2,)array, upper right cor of face frame
# eye: t*4 array, list of eyes' cor, under face frame
# parameters:
zoomout = 1 # 
ratio = .32
narrower = 0 
pns = 50
nns = 50
win = 10 # bilateral window size in pixel
print ('modules imported')

modules imported


In [2]:
#   faces eyes detection
def getEyes(frame):
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
    eye_cascade = cv.CascadeClassifier("haarcascade_eye.xml")
    if (eye_cascade.empty()==True or face_cascade.empty()==True):
        print ('Fail to load xml data')
        return []
    imgEye = []
    cnt = 0
    img = frame
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if(len(faces) != 1):
        print("Face dectection failure")
        return []
    for (x,y,w,h) in faces:
        cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),5)
        roi_gray = gray[y:y+h, x:x+w]
#         cv.imwrite('pic'+str(counter)+'.png', frame )
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            imgEye.append(cv.cvtColor(roi_color[ey+narrower:ey+(int)(ew), ex:ex+(int)(eh*zoomout)], cv.COLOR_BGR2GRAY))
            cv.putText(roi_color, str(cnt), (ex-20, ey-20), cv.FONT_HERSHEY_SIMPLEX, 3, (0,0,255),thickness=5)
            cnt += 1
    return imgEye, frame, eyes, faces[0,0:2]
print('function imported !')

function imported !


In [3]:
def showGaze(p1, p2, counter):
    imgEye[0] = cv.cvtColor(frame[p1[1]:p2[1], p1[0]:p2[0]], cv.COLOR_BGR2GRAY)
#     cv.imwrite('eye'+str(counter)+'.png', imgEye[0])

    cfR = np.array([[-1,-1,-1,1,1,1],[-1,-1,-1,-1,1,1],[-1,-1,-1,-1,-1,1],[1,1,1,1,1,1]])
    print ((imgEye[0][:,:(int)(imgEye[0].shape[1]*ratio)]).shape)
    foR = sg.convolve2d(imgEye[0][:,:(int)(imgEye[0].shape[1]*ratio)], cfR, mode='valid')
    maxpR = np.array([(int) (np.argmax(foR)/foR.shape[1]), np.argmax(foR)%foR.shape[1]])
    gradx = cv.Sobel(imgEye[0],cv.CV_64F,1,0,ksize=5)
    gradx = gradx[:,(int)(imgEye[0].shape[1]*ratio):]
    row, col = gradx.shape[0], gradx.shape[1]
    ranks = np.argsort(gradx.flatten())
    cargp = (ranks[-pns:] % col).astype('int')
    rargp = (ranks[-pns:] / col).astype('int')
    cargp += (int) ((imgEye[0].shape[1])*ratio)
    cargn = (ranks[0:nns] % col).astype('int')
    rargn = (ranks[0:nns] / col).astype('int')
    cargn += (int)((imgEye[0].shape[1])*ratio)
    cargp_, cargn_, rargp_, rargn_ = debubble(cargp, cargn, rargp, rargn, win)
    xcor = np.append(cargp_, cargn_)
    ycor = np.append(rargp_, rargn_)
    for X in xcor: 
        for Y in ycor:
            pointx = (int) (X) + p1[0]
            pointy = (int) (Y) + p1[1]
#             cv.circle(frame, (pointx, pointy), 1, (255,0,0)) 
    center, _ = efit ([xcor, ycor], maxpR)
    arre = (p1[0] + (int) (maxpR[1]), p1[1] + (int) (maxpR[0]))
    arrs = (p1[0] + (int) (center[1]), p1[1] + (int) (center[0]))
    cv.arrowedLine(frame, arrs, arre, (0,0,255), 3)
    cv.circle(frame, arrs, 3, (0,255,0)) # green RGB
    cv.circle(frame, arre, 3, (255,0,127)) # purple 
    print (maxpR[1], maxpR[0], center[0], center[1])
#     cv.imwrite('face'+str(counter)+'.png', frame )
    counter += 1
    return frame, maxpR, counter
print('function imported !')

function imported !


In [4]:
video = cv.VideoCapture(0)
counter = 0
if not video.isOpened():
    print('Could not open video!')
    sys.exit()
tracker = None
roi = (0, 0, 0, 0)
tracking_flag = -1
while True:
    key = cv.waitKey(1) & 0xff
    read_success, frame = video.read()
    if not read_success:
        print('Cannot read video file!')
        sys.exit()
    if key == 13:
        if tracking_flag == -1:
            imgEye, frame, eye, facecor = getEyes(frame)
            cv.imshow("ECE420 Lab7", frame)
            key = cv.waitKey(0) & 0xff
            if (key == 13):
                continue
            if (key-48 >= len(imgEye)): 
                print("Cannot find eye!")
                sys.exit()
            roi = eye[key-48]
            roi = tuple(roi + np.append(facecor, [0,0]))
            tracking_flag = 0
        if tracking_flag == 1:
            roi = (0, 0, 0, 0)
            tracker.clear()
            reye = None
            tracking_flag = -1
    elif key == 27:
        break
    start = cv.getTickCount()
    if tracking_flag == -1:
        cv.putText(frame, "Press ENTER to select ROI!", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    elif tracking_flag == 0:
        # Initialize KCF Tracker and Start Tracking
        # 1. Create a KCF Tracker
        # 2. Initialize KCF Tracker with grayscale image and ROI
        # 3. Modify tracking flag to start tracking
        # Your code starts here
        tracker = cv.TrackerMedianFlow_create()
        tracker.init(frame, roi)
        tracking_flag = 1
    else:

        # Update tracking result is succeed
        # If failed, print text "Tracking failure occurred!" at top left corner of the frame
        # Calculate and display "FPS@fps_value" at top right corner of the frame
        # Your code starts here
        (success, roi) = tracker.update(frame)
        if success:
            p1 = (int(roi[0]), int(roi[1]))
            p2 = (int(roi[0] + roi[2]), int(roi[1] + roi[3]))
            frame, maxpR, counter = showGaze(p1,p2, counter)
            cv.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
#             cv.imwrite('eye'+str(counter)+'.png', frame[p1[1]:p2[1], p1[0]:p2[0]])
        else:
            cv.putText(frame,"Tracking failure occured!",(10,20),cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    fps = cv.getTickFrequency() / (cv.getTickCount() - start);
    cv.putText(frame, "FPS : " + str(int(fps)), (100, 20), cv.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)
    cv.imshow("ECE420 Lab7", frame)

(69, 22)
truncation:  43 +- 10
truncation:  48 +- 10
5 29 48.190082940108034 41.628067583351424
(69, 22)
truncation:  44 +- 10
truncation:  50 +- 10
7 31 49.126591240213045 44.39626151073289
(70, 22)
truncation:  45 +- 10
truncation:  49 +- 10
7 28 49.408559185382835 43.72244231378899
(71, 22)
truncation:  45 +- 10
truncation:  49 +- 10
7 31 49.66659567100626 44.520204388388066
(71, 22)
truncation:  44 +- 10
truncation:  49 +- 10
7 31 49.40362538435001 44.330534812125016
(71, 22)
truncation:  45 +- 10
truncation:  50 +- 10
7 30 50.33093201278259 44.35776580127127
(70, 22)
truncation:  45 +- 10
truncation:  52 +- 10
7 30 50.53037971934632 43.95671861725756
(70, 22)
truncation:  44 +- 10
truncation:  50 +- 10
7 30 49.704388976354046 43.18318623967102
(71, 22)
truncation:  45 +- 10
truncation:  50 +- 10
8 32 49.54203927277286 44.126311026312415
(71, 22)
truncation:  45 +- 10
truncation:  49 +- 10
7 31 49.34912641026469 44.613136148072506
(71, 22)
truncation:  44 +- 10
truncation:  50 +- 1

13 57 56.437025054618516 48.15622057236551
(83, 26)
truncation:  52 +- 10
truncation:  56 +- 10
14 57 55.809104667616275 47.51238492158036
(83, 26)
truncation:  52 +- 10
truncation:  56 +- 10
14 57 56.302411584056635 47.41854453348029
(83, 26)
truncation:  48 +- 10
truncation:  58 +- 10
13 46 56.329142361710694 50.647144600599155
(83, 26)
truncation:  43 +- 10
truncation:  58 +- 10
13 46 55.75370658407946 47.16914246277447
(83, 26)
truncation:  42 +- 10
truncation:  59 +- 10
12 44 53.55647300764745 47.2761960995471
(83, 26)
truncation:  42 +- 10
truncation:  61 +- 10
13 46 53.31374944626672 47.24319558036849
(83, 26)
truncation:  42 +- 10
truncation:  60 +- 10
12 45 52.527337421878315 47.60249883059179
(83, 26)
truncation:  42 +- 10
truncation:  61 +- 10
13 46 53.557466098029494 48.0052212760328
(83, 26)
truncation:  42 +- 10
truncation:  61 +- 10
13 45 53.76041476779475 47.83819842280424
(83, 26)
truncation:  42 +- 10
truncation:  60 +- 10
12 45 52.93040840044931 47.64146038593634
(83

(72, 23)
truncation:  43 +- 10
truncation:  48 +- 10
8 33 47.75854585690938 43.09419594706041
(72, 22)
truncation:  42 +- 10
truncation:  47 +- 10
9 40 46.95138372372499 42.8157122813354
(71, 23)
truncation:  43 +- 10
truncation:  70 +- 10
10 39 55.91780024339837 52.25752289999279
(71, 22)
truncation:  42 +- 10
truncation:  69 +- 10
10 39 55.19561131161273 52.296460699885756
(71, 22)
truncation:  42 +- 10
truncation:  69 +- 10
11 39 55.269290363781316 52.453907269845004
(69, 22)
truncation:  66 +- 10
truncation:  66 +- 10
11 41 65.70317509319626 34.078091937739565
(68, 21)
truncation:  65 +- 10
truncation:  63 +- 10
11 41 60.024778855425254 36.59436202505077
(67, 21)
truncation:  64 +- 10
truncation:  61 +- 10
11 43 59.121383633828685 33.072911656493865
(67, 21)
truncation:  63 +- 10
truncation:  59 +- 10
8 30 57.78789191270551 32.03896692945425
(65, 21)
truncation:  63 +- 10
truncation:  59 +- 10
9 33 58.18529389556969 30.797623566623308
(65, 21)
truncation:  62 +- 10
truncation:  51 

truncation:  46 +- 10
truncation:  53 +- 10
1 33 52.08685370871719 43.56704941997904
(80, 25)
truncation:  46 +- 10
truncation:  53 +- 10
1 32 52.15444091079866 43.72610121040457
(80, 25)
truncation:  46 +- 10
truncation:  53 +- 10
1 32 52.02024960413167 43.95130023836448
(80, 25)
truncation:  46 +- 10
truncation:  53 +- 10
1 32 51.70792551641907 44.02807826924945
(80, 25)
truncation:  45 +- 10
truncation:  52 +- 10
0 32 50.84391299631501 43.81062721734744
(79, 25)
truncation:  47 +- 10
truncation:  52 +- 10
1 32 51.809584743429966 44.6524598092358
(80, 25)
truncation:  47 +- 10
truncation:  56 +- 10
1 32 52.20028215654216 45.27879091260253
(80, 25)
truncation:  47 +- 10
truncation:  27 +- 10
1 32 37.52305518283953 48.83007432809092
(80, 25)
truncation:  47 +- 10
truncation:  27 +- 10
1 32 38.33359169212676 48.280406854355704
(80, 25)
truncation:  47 +- 10
truncation:  53 +- 10
1 32 51.7592640195868 45.310598606336235
(80, 25)
truncation:  47 +- 10
truncation:  53 +- 10
9 43 51.8535982

3 34 53.25831645331205 46.323385680755166
(82, 26)
truncation:  49 +- 10
truncation:  54 +- 10
3 33 53.16796474105008 46.59718529376367
(81, 26)
truncation:  49 +- 10
truncation:  58 +- 10
3 33 55.07991008523732 45.49349306913301
(82, 26)
truncation:  50 +- 10
truncation:  39 +- 10
12 56 45.66301751814664 47.825142713384565
(82, 26)
truncation:  50 +- 10
truncation:  40 +- 10
12 43 46.07251664446879 47.32637481150429
(82, 26)
truncation:  49 +- 10
truncation:  41 +- 10
12 35 46.62855813125759 45.12905222791514
(82, 25)
truncation:  49 +- 10
truncation:  40 +- 10
12 43 45.2789876467356 44.63498942326049
(82, 26)
truncation:  49 +- 10
truncation:  40 +- 10
12 35 45.36359300697576 43.92096769039954
(83, 26)
truncation:  48 +- 10
truncation:  40 +- 10
12 35 47.396572509772895 39.72464894579251
(82, 26)
truncation:  49 +- 10
truncation:  40 +- 10
12 35 45.37043846378553 43.963447718995205
(82, 26)
truncation:  49 +- 10
truncation:  39 +- 10
12 35 46.806395177458086 39.44736638443884
(83, 26

...

truncation:  69 +- 10
truncation:  71 +- 10
12 33 69.93591368458836 46.06589978339075
(73, 23)
truncation:  70 +- 10
truncation:  67 +- 10
11 33 68.55981016411752 53.81308057553271
(72, 23)
truncation:  66 +- 10
truncation:  63 +- 10
11 33 65.79144809437629 51.40698252708006
(71, 23)
truncation:  69 +- 10
truncation:  62 +- 10
12 32 65.35997788675297 45.41052178001075
(71, 23)
truncation:  67 +- 10
truncation:  65 +- 10
12 32 65.0108442438758 40.71326183545999
(71, 22)
truncation:  65 +- 10
truncation:  65 +- 10
12 32 64.0158324575963 37.41273891330397
(70, 22)
truncation:  46 +- 10
truncation:  29 +- 10
12 32 39.34181216991364 44.60930987220226
(71, 22)
truncation:  40 +- 10
truncation:  27 +- 10
12 32 36.916363283436695 46.83567650288505
(71, 22)
truncation:  40 +- 10
truncation:  68 +- 10
13 34 53.609736381390576 30.903616632051342
(70, 22)
truncation:  36 +- 10
truncation:  67 +- 10
11 41 51.43162818395506 32.50461592740139
(71, 23)
truncation:  64 +- 10
truncation:  68 +- 10
12 38

11 65 44.46985542397628 31.409778538598005
(104, 33)
truncation:  39 +- 10
truncation:  35 +- 10
11 64 39.669740927597324 21.546309829253012
(104, 33)
truncation:  42 +- 10
truncation:  41 +- 10
23 45 42.64632601749822 21.419484923621503
(105, 33)
truncation:  52 +- 10
truncation:  33 +- 10
22 46 46.592477705274334 35.89057374681746
(104, 33)
truncation:  52 +- 10
truncation:  40 +- 10
24 45 46.60293344421904 35.170490813012044
(104, 33)
truncation:  63 +- 10
truncation:  41 +- 10
23 45 55.91252313953292 47.75527328302684
(104, 33)
truncation:  53 +- 10
truncation:  41 +- 10
24 46 45.09652463110906 37.239336919657816
(105, 33)
truncation:  41 +- 10
truncation:  41 +- 10
23 45 43.20680621827927 21.69125671103733
(104, 33)
truncation:  41 +- 10
truncation:  41 +- 10
23 46 42.82605756571377 22.231400505984293
(105, 33)
truncation:  46 +- 10
truncation:  76 +- 10
23 46 60.72165374899516 24.159305559431584
(105, 33)
truncation:  44 +- 10
truncation:  40 +- 10
23 46 45.48035061942838 34.5531

(94, 30)
truncation:  48 +- 10
truncation:  32 +- 10
7 54 47.08740786403491 36.99435243418481
(91, 29)
truncation:  78 +- 10
truncation:  33 +- 10
5 52 43.90675248971615 49.09939866805036
(90, 28)
truncation:  45 +- 10
truncation:  28 +- 10
21 33 42.13101204855609 42.11248624798247
(88, 28)
truncation:  49 +- 10
truncation:  39 +- 10
5 50 42.85772431991248 44.60224808919617
(86, 27)
truncation:  76 +- 10
truncation:  27 +- 10
7 48 51.35392721847235 43.041653938469416
(84, 27)
truncation:  74 +- 10
truncation:  27 +- 10
7 45 44.657178748138975 36.28271395803081
(82, 26)
truncation:  73 +- 10
truncation:  80 +- 10
6 45 75.16061116032355 48.752272487714514
(80, 25)
truncation:  71 +- 10
truncation:  79 +- 10
6 43 73.79207014768252 45.90278825381476
(79, 25)
truncation:  71 +- 10
truncation:  77 +- 10
6 42 71.9036289008231 45.409947037512275
(77, 24)
truncation:  69 +- 10
truncation:  75 +- 10
6 41 71.58603717037668 22.30715896835729
(76, 24)
truncation:  38 +- 10
truncation:  74 +- 10
6 3

(79, 25)
truncation:  42 +- 10
truncation:  26 +- 10
7 43 33.91642118754721 39.014087871010524
(79, 25)
truncation:  41 +- 10
truncation:  77 +- 10
9 51 58.76024157873215 25.21727002788974
(79, 25)
truncation:  41 +- 10
truncation:  26 +- 10
9 54 33.88481358940608 48.650641241596226
(79, 25)
truncation:  42 +- 10
truncation:  27 +- 10
11 52 35.79886552172882 49.25539769503698
(80, 25)
truncation:  42 +- 10
truncation:  27 +- 10
10 53 36.59959647578039 49.77898739922948
(80, 25)
truncation:  42 +- 10
truncation:  27 +- 10
11 53 36.87762277941256 50.757945308560195
(79, 25)
truncation:  43 +- 10
truncation:  27 +- 10
16 22 37.72012924473992 49.828364895088285
(78, 24)
truncation:  42 +- 10
truncation:  75 +- 10
16 22 57.74369028947764 26.011233176661705
(78, 24)
truncation:  42 +- 10
truncation:  27 +- 10
10 41 34.80956888031689 48.89433585745207
(77, 24)
truncation:  41 +- 10
truncation:  27 +- 10
12 50 34.88857676219488 48.639756153486424
(76, 24)
truncation:  40 +- 10
truncation:  74 

error: OpenCV(4.0.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
def showGazeL(p1, p2, counter): ## to be modified
    imgEye[1] = cv.cvtColor(frame[p1[1]:p2[1], p1[0]:p2[0]], cv.COLOR_BGR2GRAY)
#     cv.imwrite('eye'+str(counter)+'.png', imgEye[0])

    cfL = np.array([[1,1,1,-1,-1,-1],[1,1,-1,-1,-1,-1],[1,-1,-1,-1,-1,-1],[1,1,1,1,1,1]])
#     cfR = np.array([[-1,-1,-1,1,1,1],[-1,-1,-1,-1,1,1],[-1,-1,-1,-1,-1,1],[1,1,1,1,1,1]])
    print ((imgEye[0][:,:(int)(imgEye[0].shape[1]*ratio)]).shape)
    foR = sg.convolve2d(imgEye[0][:,:(int)(imgEye[0].shape[1]*ratio)], cfR, mode='valid')
    maxpR = np.array([(int) (np.argmax(foR)/foR.shape[1]), np.argmax(foR)%foR.shape[1]])
    foL = sg.convolve2d(imgEye[1][:,:(int)()])
    
    gradx = cv.Sobel(imgEye[0],cv.CV_64F,1,0,ksize=5)
    gradx = gradx[:,(int)(imgEye[0].shape[1]*ratio):]
    row, col = gradx.shape[0], gradx.shape[1]
    ranks = np.argsort(gradx.flatten())
    cargp = (ranks[-pns:] % col).astype('int')
    rargp = (ranks[-pns:] / col).astype('int')
    cargp += (int) ((imgEye[0].shape[1])*ratio)
    cargn = (ranks[0:nns] % col).astype('int')
    rargn = (ranks[0:nns] / col).astype('int')
    cargn += (int)((imgEye[0].shape[1])*ratio)
    cargp_, cargn_, rargp_, rargn_ = debubble(cargp, cargn, rargp, rargn, win)
    xcor = np.append(cargp_, cargn_)
    ycor = np.append(rargp_, rargn_)
    for X in xcor: 
        for Y in ycor:
            pointx = (int) (X) + p1[0]
            pointy = (int) (Y) + p1[1]
#             cv.circle(frame, (pointx, pointy), 1, (255,0,0)) 
    center, _ = efit ([xcor, ycor], maxpR)
    arre = (p1[0] + (int) (maxpR[1]), p1[1] + (int) (maxpR[0]))
    arrs = (p1[0] + (int) (center[1]), p1[1] + (int) (center[0]))
    cv.arrowedLine(frame, arrs, arre, (0,0,255), 3)
    cv.circle(frame, arrs, 3, (0,255,0)) # green RGB
    cv.circle(frame, arre, 3, (255,0,127)) # purple 
    print (maxpR[1], maxpR[0], center[0], center[1])
#     cv.imwrite('face'+str(counter)+'.png', frame )
    counter += 1
    return frame, maxpR, counter
print('function imported !')

In [ ]:
def mousePos():
    if event == cv.event_lbuttondbclk:
        